### Homework #1: Linear Regression (maximum 10 points)

Some tasks will have different variants (there are 4 in total).
To find out your variant, count the number of letters in your last name, take the remainder after dividing by 4, and add 1.

In [1]:
surname = "Khachatryan"
letters_count = len(surname)
variant = (letters_count % 4) + 1
variant

4

In [2]:
import numpy as np

#### Multivariate Linear Regression from sklearn

Let’s create a dataset for multivariate regression.

In [3]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


We have 10,000 observations and 100 features.
First, let’s solve the problem analytically “out of the box.”

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

4.517260350543703e-25


Now let’s train linear regression using gradient descent, also “out of the box.”

In [5]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.00000001).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))
reg.coef_

4.464757542693889e-12


array([ 6.00705111e-08,  6.30153237e+01,  3.48735829e-08,  3.04924606e-09,
        4.94863598e-08,  1.40682365e-08,  2.44767416e-08, -1.03822729e-08,
       -5.30425967e-09, -3.85841742e-09,  5.25769499e-09, -7.94299818e-09,
        3.11430378e-08,  5.44139079e-08,  8.39339973e+01,  6.13312826e-09,
        1.31004561e-09, -5.29333391e-08, -4.15935652e-09,  1.64836351e-08,
       -5.67348134e-09,  3.01089759e-08,  2.58541725e-08,  6.62792724e+01,
       -5.96686586e-08,  8.33654736e+01,  3.75923050e-09,  5.33468026e-08,
        3.07703726e-08, -3.99931084e-08,  8.38706540e-09,  1.22952100e-08,
       -4.43324677e-08, -1.27071475e-08,  5.31720735e-08,  2.29093276e-08,
       -8.39955047e-09,  6.38160414e-08,  6.75280726e-08, -3.35819450e-09,
       -5.29295645e-09,  3.81422983e-08,  2.40703297e-08, -2.46318298e-09,
        9.83993128e+01, -6.29936943e-09,  3.62150781e-08,  6.25566974e-08,
        7.52658664e+00,  5.65939652e+01, -6.31275082e-09, -2.31639079e-08,
        9.07083661e-09, -

#### Task 1 (1 point)

Explain why there is a difference between the two MSE values obtained.

LinearRegression uses analytical OLS, which finds exact optimal weights (coefficients) for the linear model by minimizing MSE on the whole dataset.

SGDRegressor uses stochastic gradient descent, which updates weights step by step using small data batches on each iteration.
This is an approximate and noisier method, so the resulting coefficients might be less accurate (or sometimes more accurate, depending on randomness).
The learning rate can also be too high or too low, affecting convergence.
Additionally, the alpha parameter adds a penalty for large coefficients.
Because of the stochastic nature of the method, the process may fluctuate depending on random initialization and sample order.

Unlike LinearRegression, which computes exact weights using all data at once, SGDRegressor updates weights sequentially, which introduces randomness and may cause variation between runs.

In my case, LinearRegression performs better, although in some runs SGDRegressor produced lower MSE.

#### Task 2 (1 point)

Tune the hyperparameters of gradient descent so that the MSE value is close to that obtained with LinearRegression

In [6]:
# I tried many options — this seems to produce the closest result
reg = SGDRegressor(
    alpha = 1e-14,                 # stronger regularization
    max_iter = 1000,               # leave iteration count as default
    learning_rate = 'constant',    # constant learning rate
    eta0 = 0.001                   # initial learning step size
)
reg.fit(X, y)

print(mean_squared_error(y, reg.predict(X)))
reg.coef_

3.676095881190452e-25


array([ 2.72215222e-14,  6.30153243e+01,  7.64638772e-15,  1.34819691e-14,
       -4.02234865e-17, -1.04094146e-14,  1.57234496e-14, -1.36968636e-14,
       -1.13847359e-14, -3.25267114e-14, -2.54187969e-15,  2.74503173e-15,
        2.70146305e-15,  2.65315043e-14,  8.39339983e+01, -1.09583998e-14,
       -1.02700313e-14, -6.53081467e-15, -2.93141706e-14, -2.00142969e-14,
        9.15582152e-15,  5.77986090e-15,  1.54796365e-14,  6.62792731e+01,
       -3.77909379e-14,  8.33654745e+01, -4.31592502e-15,  1.27500124e-16,
        2.28885878e-14, -1.75751666e-15,  4.47360875e-15,  1.24668258e-14,
       -1.24097430e-14, -2.03990656e-14, -1.01733905e-14, -1.37744957e-14,
       -1.02653952e-14,  2.43652618e-15,  5.79282242e-15,  2.28014899e-14,
        6.35821370e-15, -1.13771680e-14, -1.44598225e-15, -1.05201924e-14,
        9.83993137e+01, -1.03812664e-14,  1.54788612e-14,  5.19949089e-15,
        7.52658676e+00,  5.65939657e+01, -2.07395433e-14, -1.99351764e-14,
       -9.86396008e-15,  

#### Your Own Multivariate Linear Regression

# I used ChatGPT to optimize the code here :)
# I used to calculate it with a loop over features
class MyLinearRegression:
    def fit(self, X, y):
        # add column of ones for intercept
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        # analytical solution using normal equations
        self.coef_ = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y
        
    def predict(self, X):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        return X_b @ self.coef_

Now check how our model performs.

In [8]:
my_reg = MyLinearRegression()
my_reg.fit(X, y)
print(mean_squared_error(y, my_reg.predict(X)))

1.5123811161210144e-26


#### Task 3 (1 point)

Compare the coefficients of your model with those from LinearRegression.

In [9]:
lin_reg = LinearRegression().fit(X, y)

np.allclose(
    my_reg.coef_,
    np.r_[lin_reg.intercept_, lin_reg.coef_]
)

True

Explanation:
Our implementation is mathematically equivalent to LinearRegression because both use the same analytical ordinary least-squares solution via normal equations.

#### Task 4 (1 point)

Add normalization of features before model training.

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

reg_scaled = LinearRegression().fit(X_scaled, y)
print(mean_squared_error(y, reg_scaled.predict(X_scaled)))


4.470862475281235e-25


#### Task 5 (1 point)

Implement normalization inside your own class.

In [12]:
class MyLinearRegressionScaled:
    def fit(self, X, y):
        self.mean_ = X.mean(axis=0)
        self.std_ = X.std(axis=0)
        X_scaled = (X - self.mean_) / self.std_
        X_b = np.c_[np.ones((X_scaled.shape[0], 1)), X_scaled]
        self.coef_ = np.linalg.inv(X_b.T @ X_b) @ X_b.T @ y

    def predict(self, X):
        X_scaled = (X - self.mean_) / self.std_
        X_b = np.c_[np.ones((X_scaled.shape[0], 1)), X_scaled]
        return X_b @ self.coef_

In [13]:
reg_my_scaled = MyLinearRegressionScaled()
reg_my_scaled.fit(X, y)
print(mean_squared_error(y, reg_my_scaled.predict(X)))

2.2328312664263254e-26


#### Task 6 (1 point)

Compare your normalized model with the one from sklearn.

In [14]:
reg_sk_scaled = LinearRegression().fit(X_scaled, y)
np.allclose(
    reg_my_scaled.coef_,
    np.r_[reg_sk_scaled.intercept_, reg_sk_scaled.coef_]
)

True

Explanation:
Normalization doesn’t change model quality but makes optimization more stable and interpretable — all coefficients are on a comparable scale.

#### Task 7 (1 point)

Generate a dataset with noise and analyze performance.

In [15]:
X_noisy, y_noisy = make_regression(
    n_samples=1000, n_features=10, noise=50, random_state=42
)
reg_noise = LinearRegression().fit(X_noisy, y_noisy)
print(mean_squared_error(y_noisy, reg_noise.predict(X_noisy)))

2321.882005350111


Conclusion:
Adding noise increases MSE because the target variable now contains random components that the model can’t explain.

#### Task 8 (1 point)
Add l1 (for the first and fourth variants) or l2 (for the second and third variants) regularization.

In [16]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error

class LinearRegressionGD:
    def __init__(self, alpha=0.01, tol=1e-6, max_iter=1000, l1_ratio=0.01):
        self.alpha = alpha
        self.tol = tol
        self.max_iter = max_iter
        self.l1_ratio = l1_ratio 

    def fit(self, X, y):
        X_b = np.c_[np.ones((X.shape[0], 1)), X] 
        self.theta = np.zeros(X_b.shape[1])      

        for i in range(self.max_iter):
            gradients = 2 / X_b.shape[0] * X_b.T.dot(X_b.dot(self.theta) - y)  
            
            l1_penalty = self.l1_ratio * np.sign(self.theta)
            gradients += l1_penalty

            new_theta = self.theta - self.alpha * gradients                   

            if np.linalg.norm(new_theta - self.theta, ord=2) < self.tol:
                break

            self.theta = new_theta

    def predict(self, X):
        X_b = np.c_[np.ones((X.shape[0], 1)), X] 
        return X_b.dot(self.theta)  

if __name__ == "__main__":
    X, y = make_regression(n_samples=10000, n_features=100, noise=0.1, random_state=42)

    model = LinearRegressionGD(alpha=0.01, max_iter=1000, tol=1e-6, l1_ratio=0.01)
    model.fit(X, y)

    predictions = model.predict(X)

    mse = mean_squared_error(y, predictions)
    print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.01027350190221907


#### Task 9 (1 point)


In [18]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.1)
lasso.fit(X, y)
print(f"Mean Squared Error {mean_squared_error(y, lasso.predict(X))}")

Mean Squared Error 0.1113104172641727
